In [1]:
import requests
import io

from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
c.head(3)

,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2019-12-31,Afghanistan,0,0,0,0
1,2020-01-01,Afghanistan,0,0,0,0
2,2020-01-02,Afghanistan,0,0,0,0


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv")

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-05,France,132226.0,0.0,25449.0,0.0,0.0,0.0
0,2020-05-05,China,84051.0,0.0,4664.0,0.0,0.0,0.0
0,2020-05-05,Italy,213292.0,0.0,29240.0,0.0,0.0,0.0
0,2020-05-05,Spain,218700.0,0.0,25556.0,0.0,0.0,0.0
0,2020-05-05,United States,1206402.0,0.0,70150.0,0.0,0.0,0.0
0,2020-05-05,World,3627046.0,0.0,254527.0,0.0,0.0,0.0
0,2020-05-05,United Kingdom,195005.0,0.0,28954.0,0.0,0.0,0.0
0,2020-05-05,Germany,164272.0,0.0,6915.0,0.0,0.0,0.0
0,2020-05-05,Iran,99643.0,0.0,6333.0,0.0,0.0,0.0
0,2020-05-05,Turkey,129021.0,0.0,3509.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")

In [6]:
## concatenate all df in 1
frames = [follow_df1,follow_df2,follow_df3,follow_df4,follow_df5,follow_df6,follow_df7,follow_df8,follow_df9,follow_df10]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-06,France,132226.0,132967.0,25449.0,25531.0,-741.0,-82.0
0,2020-05-06,China,84051.0,83968.0,4664.0,4637.0,83.0,27.0
0,2020-05-06,Italy,213292.0,213013.0,29240.0,29315.0,279.0,-75.0
0,2020-05-06,Spain,218700.0,219329.0,25556.0,25613.0,-629.0,-57.0
0,2020-05-06,United States,1206402.0,1204475.0,70150.0,71078.0,1927.0,-928.0
0,2020-05-06,World,3627046.0,3623803.0,254527.0,256880.0,3243.0,-2353.0
0,2020-05-06,United Kingdom,195005.0,194990.0,28954.0,29427.0,15.0,-473.0
0,2020-05-06,Germany,164272.0,164897.0,6915.0,6996.0,-625.0,-81.0
0,2020-05-06,Iran,99643.0,99970.0,6333.0,6340.0,-327.0,-7.0
0,2020-05-06,Turkey,129021.0,129491.0,3509.0,3520.0,-470.0,-11.0


In [7]:
## dump dataset for use it in other nb
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")